# Detecting Packing with Yara
* Install Yara-python
* Go to https://github.com/Yara-Rules/rules and download rules
* Go to https://yara.readthedocs.io/en/latest/yarapython.html for documentation

In [ ]:
!pip install yara-python
!pip install pefile

In [ ]:
import yara
from os import listdir
import os.path
import pefile

In [ ]:
rules_path = '../input/yararules/'
peid_rules = yara.compile(rules_path + 'peid.yar')
packer_rules = yara.compile(rules_path + 'packer.yar')
crypto_rules = yara.compile(rules_path + 'crypto.yar')
antidebug_rules = yara.compile(rules_path + 'antidebug.yar')

In [ ]:

packers_sections = {
        #The packer/protector/tools section names/keywords
        '.aspack': 'Aspack packer',
        '.adata': 'Aspack packer/Armadillo packer',
        'ASPack': 'Aspack packer',
        '.ASPack': 'ASPAck Protector',
        '.boom': 'The Boomerang List Builder (config+exe xored with a single byte key 0x77)',
        '.ccg': 'CCG Packer (Chinese Packer)',
        '.charmve': 'Added by the PIN tool',
        'BitArts': 'Crunch 2.0 Packer',
        'DAStub': 'DAStub Dragon Armor protector',
        '!EPack': 'Epack packer',
        'FSG!': 'FSG packer (not a section name, but a good identifier)',
        '.gentee': 'Gentee installer',
        'kkrunchy': 'kkrunchy Packer',
        '.mackt': 'ImpRec-created section',
        '.MaskPE': 'MaskPE Packer',
        'MEW': 'MEW packer',
        '.MPRESS1': 'Mpress Packer',
        '.MPRESS2': 'Mpress Packer',
        '.neolite': 'Neolite Packer',
        '.neolit': 'Neolite Packer',
        '.nsp1': 'NsPack packer',
        '.nsp0': 'NsPack packer',
        '.nsp2': 'NsPack packer',
        'nsp1': 'NsPack packer',
        'nsp0': 'NsPack packer',
        'nsp2': 'NsPack packer',
        '.packed': 'RLPack Packer (first section)',
        'pebundle': 'PEBundle Packer',
        'PEBundle': 'PEBundle Packer',
        'PEC2TO': 'PECompact packer',
        'PECompact2': 'PECompact packer (not a section name, but a good identifier)',
        'PEC2': 'PECompact packer',
        'pec1': 'PECompact packer',
        'pec2': 'PECompact packer',
        'PEC2MO': 'PECompact packer',
        'PELOCKnt': 'PELock Protector',
        '.perplex': 'Perplex PE-Protector',
        'PESHiELD': 'PEShield Packer',
        '.petite': 'Petite Packer',
        'petite': 'Petite Packer',
        '.pinclie': 'Added by the PIN tool',
        'ProCrypt': 'ProCrypt Packer',
        '.RLPack': 'RLPack Packer (second section)',
        '.rmnet': 'Ramnit virus marker',
        'RCryptor': 'RPCrypt Packer',
        '.RPCrypt': 'RPCrypt Packer',
        '.seau': 'SeauSFX Packer',
        '.sforce3': 'StarForce Protection',
        '.spack': 'Simple Pack (by bagie)',
        '.svkp': 'SVKP packer',
        'Themida': 'Themida Packer',
        '.Themida': 'Themida Packer',
        'Themida ': 'Themida Packer',
        '.taz': 'Some version os PESpin',
        '.tsuarch': 'TSULoader',
        '.tsustub': 'TSULoader',
        '.packed': 'Unknown Packer',
        'PEPACK!!': 'Pepack',
        '.Upack': 'Upack packer',
        '.ByDwing': 'Upack Packer',
        'UPX0': 'UPX packer',
        'UPX1': 'UPX packer',
        'UPX2': 'UPX packer',
        'UPX!': 'UPX packer',
        '.UPX0': 'UPX Packer',
        '.UPX1': 'UPX Packer',
        '.UPX2': 'UPX Packer',
        '.vmp0': 'VMProtect packer',
        '.vmp1': 'VMProtect packer',
        '.vmp2': 'VMProtect packer',
        'VProtect': 'Vprotect Packer',
        '.winapi': 'Added by API Override tool',
        'WinLicen': 'WinLicense (Themida) Protector',
        '_winzip_': 'WinZip Self-Extractor',
        '.WWPACK': 'WWPACK Packer',
        '.yP': 'Y0da Protector',
        '.y0da': 'Y0da Protector',
    }

In [ ]:
packers_sections_lower =  {x.lower(): x for x in packers_sections.keys()}
def detect_packing(sections_of_pe):
    return [packers_sections_lower[x.lower()] for x in sections_of_pe if x.lower() in packers_sections_lower.keys()]
def cast_clear(x):
    if '->' in x: 
        return x
    else:
        return 'clear'

In [ ]:
packers = ['AHTeam', 'Armadillo', 'Stelth', 'yodas', 'ASProtect', 'ACProtect', 'PEnguinCrypt', 
 'UPX', 'Safeguard', 'VMProtect', 'Vprotect', 'WinLicense', 'Themida', 'WinZip', 'WWPACK',
 'Y0da', 'Pepack', 'Upack', 'TSULoader'
 'SVKP', 'Simple', 'StarForce', 'SeauSFX', 'RPCrypt', 'Ramnit', 
 'RLPack', 'ProCrypt', 'Petite', 'PEShield', 'Perplex',
 'PELock', 'PECompact', 'PEBundle', 'RLPack', 'NsPack', 'Neolite', 
 'Mpress', 'MEW', 'MaskPE', 'ImpRec', 'kkrunchy', 'Gentee', 'FSG', 'Epack', 
 'DAStub', 'Crunch', 'CCG', 'Boomerang', 'ASPAck', 'Obsidium','Ciphator',
 'Phoenix', 'Thoreador', 'QinYingShieldLicense', 'Stones', 'CrypKey', 'VPacker',
 'Turbo', 'codeCrypter', 'Trap', 'beria', 'YZPack', 'crypt', 'crypt', 'pack',
 'protect', 'tect'
]

In [ ]:
base_dir= '../input/secondpart/samples1 2019'
columns = ['Hash of Malware', 'Status', 'Type']
rows = []
counter = 0
for f in listdir(base_dir):
    if not f.endswith('.json'):
        print(counter)
        counter += 1 
        try:
            matches = crypto_rules.match(os.path.join(base_dir,f))
            if matches:
                rows.append([f,'crypted',matches[0]])
                continue
        except:
            print('crypto exception')
        try:
            matches = packer_rules.match(os.path.join(base_dir,f))
            if matches:
                rows.append([f,'packed',matches[0].lower()])
                continue
        except:
            print('packer exception')
        try:
            exe = pefile.PE(os.path.join(base_dir,f), fast_load=True)
            matches = detect_packing([
                section.Name.decode(errors='replace',).rstrip('\x00') for section in exe.sections
            ])
            if matches:
                rows.append([f,'packed',matches[0].lower()])
                continue
        except:
            print('manuel exception')
        try:
            matches = peid_rules.match(os.path.join(base_dir,f))
            if matches:
                for packer in packers:
                    for match in matches:
                        if packer.lower() in match.lower():
                            rows.append([f, 'packed', match.lower()])
                            continue
        except:
            print('peid exception')
        rows.append([f, 'clear', 'None'])    
         

In [ ]:
from collections import Counter

In [ ]:
Counter([x[1] for x in rows])

In [ ]:
import pandas as pd

In [ ]:
data_frame = pd.DataFrame(rows, columns=columns)

In [ ]:
data_frame.head()

In [ ]:
data_frame.to_csv('malware_list_second_part.csv')